<a href="https://colab.research.google.com/github/Sergey-Kiselev-dev/NN_01_Keras/blob/main/NN_01_02_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Первая нейронная сеть на Keras
===
Сегодня обсудим:

1. Как создать нейронную сеть в Keras
2. Как обучить нейронную сеть в Keras

И сегодня обучим три нейронные сети для решения задач:

1. Нейрон для умножения
2. Нейросеть для сложения
3. Нейросеть для классификации изображений

Давайте создадим нейронную сеть, состоящую из одного нейрона.

input --> 0 --> output

1. Sequential - это класс последовательности слоев в нейронной сети, у нас пока будет только один слой, но сюда можно добавлять сколь угодно слоев и это будет сеть, состоящая из последовательности слоев.
2. Dense - это класс полносвязного/линейного слоя, все нейроны связаны друг с другом.
3. units=1 - это количество нейронов в слое. У нас 1 нейрон.
4. input_shape=(1,) - это входная размерность объекта. У нас только 1 вход.
5. activation='relu' - это функция активации, которая добавляет в слой нелинейности, именно из-за неё мы можем получать более сложные результаты работы сети.

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential([
    Dense(1, input_shape=(1,), activation='relu')
])

model.summary()

In [ ]:
# for reproducible weights initialization
import tensorflow as tf
tf.random.set_seed(1)

In [ ]:
import tensorflow as tf
tf.random.set_seed(1)

model = Sequential([
    Dense(1, input_shape=(1,), activation='relu')
])

model.get_weights()

In [ ]:
import numpy as np

X = np.array([[1], [3], [2], [10], [4], [7], [8]])
y = np.array([[3, 9, 6, 30, 12, 21, 24]]).T

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential([
    Dense(1, input_shape=(1,), activation='linear')
])

model.summary()

In [ ]:
w1, w0 = model.get_weights()
w1, w0

In [ ]:
X[:-1]

In [ ]:
model.predict(X[:1])

In [ ]:
w1 * X[:1] + w0

In [ ]:
from keras.activations import linear
linear(w1 * X[:1] + w0)

In [ ]:
import keras
model.compile(optimizer='sgd', loss='mse', metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
%%time
model.fit(X, y, epochs=100)

In [ ]:
user_inp1, user_inp2 = 5, -9
print(f"Проверка на новых данных: {user_inp1} {user_inp2}")
print("Предсказание нейронной сети: ")
print(model.predict(np.array([[user_inp1], [user_inp2]])))

In [ ]:
nw1, nw0 = model.get_weights()
print('w1 before', w1, 'w1 after', nw1)
print('w0 before', w0, 'w0 after', nw0)

In [ ]:
import pandas as pd

pd.DataFrame({
   'true': np.squeeze(y),
   'pred': np.squeeze(model.predict(X))
})

Сеть для сложения чисел
===========

In [ ]:
X1 = np.random.randint(1, 10, size=50)
X2 = np.random.randint(1, 10, size=50)

y = X1 + X2

In [ ]:
X = np.vstack([X1, X2]).T
X

In [ ]:
y = y[None]
y = y.T
y

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_norm = mms.fit_transform(X)

Для того, что обучить нейронную сеть для любой задачи нужно ответить на три вопроса:

1. Какая архитектура сети?

2. Что оптимизируем?

3. Как обучаем?

Какая архитектура сети
------------
Создадим сеть посложнее, она будет состоять уже из двух слоев, чтобы быстрее обучалась. В одном слое 3 нейрона, а в выходном слое 1 нейрон.

input_1 --> 0 -->

            0 --> 0 Output

input_2 --> 0 -->

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
tf.random.set_seed(9)

model = Sequential([
    Dense(3, input_shape=(2,), activation='linear'),
    Dense(1, activation='linear')
])

model.summary()

Что здесь происходит с весами и с самой архитектурой?

Количество весов для одного нейрона равно 2, т.к. два входа поступает, а вдобавок у каждого нейрона есть 1 bias, на первом слое 3 нейрона, значит суммарно весов на первом слое будет 9.

Второй слой ждет на вход 3 сигнала, еще 1 bias.

Получаем 9 + 4 = 13 обучаемых весов.

In [ ]:
model.get_weights()

Что оптимизируем
---
Снова задача регрессии, поэтому берем функцию потерь MSE.

Как оптимизируем
---
Возьмем тот же самый градиентный спуск со стохастикой.

In [ ]:
model.compile(optimizer='sgd', loss='mse', metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
%%time
model.fit(X_norm, y, epochs=200)

Сеть обучается, ошибка падает, метрика становится лучше, всё замечательно.

Теперь проверим, а как модель работает на новых данных.

In [ ]:
test_X = [[4, 2],
          [6, 2]]
test_X = mms.transform(test_X)
print("Предсказание нейронной сети: ")
print(model.predict(np.array(test_X)))

Предсказания очень похожи на истину.

И проверимся на всех обучающих данных.

In [ ]:
import pandas as pd

pd.DataFrame({
    'x1': X[:, 0],
    'x2': X[:, 1],
    'true': np.squeeze(y),
    'pred': np.squeeze(model.predict(X_norm))
}).head(10)

Сеть для классификации изображений
===
Подгрузим данные из стандартных датасетов из keras.

Датасет называется MNIST и представляет из себя черно-белые изображения 28 на 28 пикселей.

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, X_test.shape

In [ ]:
X_train[0].shape

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 5, figsize=(15, 10))

for i in range(5):
    ax[i].imshow(X_train[i], cmap='gray')
    ax[i].axis('off')

In [ ]:
y_train[:5]

Но давайте упростим себе задачу и возьмем для обучения только два класса, чтобы сделать задачу бинарной классификации.

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
idxs = np.where((y_train == 0) | (y_train == 1))
idxs

In [ ]:
idxs = np.where((y_train == 0) | (y_train == 1))
y_train = y_train[idxs]
X_train = X_train[idxs]
X_train.shape, y_train.shape

И тоже самое для теста.

In [ ]:
idxs = np.where((y_test == 0) | (y_test == 1))
y_test = y_test[idxs]
X_test = X_test[idxs]
X_test.shape, y_test.shape

Убедимся, что теперь у нас только 0, либо 1

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15, 10))

for i in range(5):
    ax[i].imshow(X_train[i], cmap='gray')
    ax[i].axis('off')

In [ ]:
y_train[:5]

Нормируем данные, сейчас обойдемся без MinMaxScaler из sklearn, а воспользуемся делением на 255, т.к. сейчас изображения представлены пикселями в диапазоне от 0 до 255, а для нейросети комфортней обучаться на диапазоне от 0 до 1.

In [ ]:
print(X_train.min(), X_train.max())

X_train = X_train / 255.0
X_test = X_test / 255.0

print(X_train.min(), X_train.max())

Так же нужно видоизменить метку класса, сейчас это лейблы 0 или 1, нужно преобразовать в бинарный вид.

Тем самым получаем 2 столбика, где первый - это метка является ли изображение 0 классом, а второй столбик - является ли изображение 1 классом.

In [ ]:
from keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

y_train[:5]

А чтобы еще легче обучать сетку поменяем масштаб изображений, сейчас они 28 на 28, сделаем меньше, чтобы нейросеть была легче.

In [ ]:
X_train[..., np.newaxis].shape

In [ ]:
import matplotlib.pyplot as plt

X_train_resized = tf.image.resize(X_train[..., np.newaxis], (6, 6))[..., 0]
X_test_resized = tf.image.resize(X_test[..., np.newaxis], (6, 6))[..., 0]

fig, ax = plt.subplots(1, 5, figsize=(15, 10))

for i in range(5):
    ax[i].imshow(X_train_resized[i], cmap='gray')
    ax[i].axis('off')

Для того, что обучить нейронную сеть для любой задачи нужно ответить на три вопроса:

1. Какая архитектура сети?
2. Что оптимизируем?
3. Как обучаем?

Какая архитектура сети
---
Создадим сеть еще сложнее.

Во-первых, на вход поступает изображение 6х6, нужно с ним что-то сделать, так как наша сетку пока не умеет работать с двумерным входом. Здесь нам поможет слой из keras Flatten, который вытягивает изображение в один вектор, была картинка 6x6, а станет вектором с размерностью 36.

Была матрица:

In [ ]:
X_train_resized[0].numpy()

In [ ]:
X_train_resized[0].numpy().shape

А теперь вектор:

In [ ]:
X_train_resized[0].numpy().flatten()

In [ ]:
X_train_resized[0].numpy().flatten().shape

Во-вторых, на выходе не что-то одно, а две вероятности быть или не быть определенным классом.

А значит на выходе имеем два нейрона, каждый из которых отвечает за класс.

input_1 -->

input_2 -->  0  proba 0

imput_3 -->  1  proba 1

...      

input_36 -->

В-третьих, на выходном слое нужно использовать другую функцию активации, а именно sigmoid, так как она позволяет решать задачу бинарной классификации очень хорошо.

In [ ]:
from keras.layers import Flatten
tf.random.set_seed(9)

model = Sequential([
    Flatten(input_shape=(6, 6)),
    Dense(2, activation='sigmoid')
])

model.summary()

Считаем количество весов.

Для одного нейрона - 36 входов, плюс 1 bias. Для второго нейрона тоже самое.

А значит, (36 + 1) * 2 = 74 настраиваемых весов.

Что оптимизируем
---
У нас задача бинарной классификации, поэтому берем функцию потерь, которая подходит сюда.

Это к примеру, бинарная кросс-энтропия.

А еще будем считать метрику классификации accuracy, если требуется вспомнить метрики качества для классификации, то советую вам посмотреть данный плейлист.

Как оптимизируем
---
Возьмем тот же самый градиентный спуск со стохастикой.

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
%%time
model.fit(X_train_resized, y_train_cat, epochs=5)

Сеть обучается, ошибка падает, метрика становится лучше, всё замечательно.

Теперь проверим, а как модель работает на новых данных.

На выходе модель дает 2 вероятности:

1. Быть нулевым классом
2. Быть первым классом

Для выбранного объекта вероятность быть первым классом гораздо выше, чем вероятность быть нулевым классом.

In [ ]:
print("Предсказание нейронной сети: ")
pred = model.predict(X_test_resized[:1])
pred

Для того, чтобы выдать финальную метку класса можем взять класс, где максимальная предсказанная вероятность.

In [ ]:
pred_cls = pred.argmax()
pred_cls

Давайте проверим предсказание визуально.

In [ ]:
idx = 0
plt.imshow(X_test_resized[idx])
plt.title(f'pred {pred_cls}, true {y_test[idx]}');

А это как выглядило изображение до изменения размера.

In [ ]:
plt.imshow(X_test[idx])
plt.title(f'pred {pred_cls}, true {y_test[idx]}');

И проверимся на всех обучающих данных.

Делаем предсказания на всех тестовых объектах.

In [ ]:
preds = model.predict(X_test_resized)
preds

И берем метку класса, где максимальная вероятность.

In [ ]:
preds_cls = preds.argmax(axis=1)
preds_cls

И можем посчитать метрику качества.

In [ ]:
from sklearn.metrics import accuracy_score

print(f'test acc: {accuracy_score(y_test, preds_cls)*100:.2f}% ({(y_test == preds_cls).sum()} out of {y_test.shape[0]})')

Вот так вот мы и обучили сетку на задачу классификации.

А на самом деле уже обучили целых три нейронных сети за одно занятие.

Практика
---
Практика доступна на платформе boosty https://boosty.to/machine_learrrning/posts/f0bea364-3056-4c84-96d0-5586dea90d72

Доступна

1. по подписке уровня light+ и выше
2. разовая оплата

Summary
---
Вот мы и разобрались, как обучается нейронная сеть с помощью keras.

Получаем выборку для обучения
---
1. масштабирование данных (MinMaxScaler, StandardScaler, /255.0)
2. Resize данных при необходимости
3. Для классификации нужно перевести метки классов в бинарное представление

Создаем архитектуру
---
1. выбираем количество входов
2. выбираем количество слоев
3. выбираем количество выходных нейронов
4. выбираем функцию активации

Что нужно оптимизировать
---
1. выбор функции потерь
2. из стандартных и привычных
для регрессии - MSE
для классификации - binary_crossentropy, categorical_crossentropy

Как нужно оптимизировать
---
1. выбор оптимизатора
2. из стандартных и привычных: sgd, adam

Компиляция модели .compile()
---
Обучение модели .fit()
---
Проверка результатов
---
